# Data Processing to Identify Suitable Offshore Renewable Energy (ORE) Development Locations

## Summary
This script takes shapefiles for geoprocessing to identify the most suitable ORE development locations. This is done by taking a baseline shapefile, such as bathymetry, and clipping out constraints that will result in a "Suitability Area" shapefile that can be mapped in a viewer, such as Terria. Any shapefile that requires a buffer, can be processed using our "Shapefile Buffer" script.

In this example, we use the Irish data and clip bathymetry by the following constraints, according to our National Marine Planning policy:
- Depth (>150m for floating and 80m for offshore)
- Ferry Routes (with 5km buffer)
- Shipwrecks (with 500m buffer)
- Special Areas of Conservation
- Special Protected Areas
- Defence Restriction Zones

The shapefiles  you are using are imported from S3 storage and output to S3 for inclusion in your web viewer. If you are pulling spatial data from a public Feature server, we recommend using our "Feature to Shapefile" notebook in this folder to query and download your feature service layer as a shapefile to S3 storage. You can also pull from an Image server and convert image to raster for analysis and save as a shapefile as we have done with our bathymetry layer in "Image to Shapefile".

## Data import
It is presumed that you have imported all required files to your S3 storage bucket as a shapefile. 

If pulling from a Feature server, use "Feature to Shapefile" and apply relevant buffers using "Shapefile Buffer". If pulling from an Image server, you can see "Image to Shapefile" where we have performed clipping on our initial bathymetry layer. Use "Shapefile_Buffer" to apply a buffer to any contraint shapefiles if required.

In [ ]:
### Take required files from S3 ###

bathymetry = "INFOMAR-150.shp" 
ferry_routes_buffer = "Ferry_Routes_Buffer.shp" #Test with buffer script
shipwrecks_buffer = "Shipwrecks_Buffer.shp" # Test with buffer script
SAC = "Special_Area_Conservation.shp"
SPA = "Special_Protected_Area.shp"
restricted_zones = "Danger and Restricted areas that coincide with Marine or Coastal Areas only.shp"

In [ ]:
upload_model_to_s3_bucket()

## Data analysis

In [4]:
!pip install geopandas

In [ ]:
def clip_shapefiles (shapefile_a, shapefile_b, output_file):
    """
    Subtracts the geometry of shapefile B from shapefile A and saves the result.

    Parameters
    ----------
    shapefile_a : str
        Path to the input shapefile A.
    shapefile_b : str
        Path to the shapefile whose geometries will be subtracted from A.
    output_file : str, optional
        Path to save the resulting shapefile. Default is 'Clipped.shp'.

    Returns
    -------
    geopandas.GeoDataFrame
        The resulting GeoDataFrame after subtraction.
    """
    # Load shapefiles
    gdf_a = gpd.read_file(shapefile_a)
    gdf_b = gpd.read_file(shapefile_b)

    #print("A CRS:", gdf_a.crs)
    #print("B CRS:", gdf_b.crs)

    #if gdf_b.crs is None:
    #    gdf_b = gdf_b.set_crs(epsg=3857)

    # Align CRS
    gdf_b = gdf_b.to_crs(gdf_a.crs)

    # Dissolve B into a single geometry
    b_union = gdf_b.unary_union

    # Subtract geometries
    gdf_a['geometry'] = gdf_a.geometry.apply(lambda geom: geom.difference(b_union))

    # Remove empty geometries
    gdf_result = gdf_a[~gdf_a.is_empty]

    # Save result
    gdf_result.to_file(output_file) ################################################################## SAVE TO S3 #####################################################################################
    print(f"New shapefile '{output_file}' created successfully.") 

    return gdf_result


In [ ]:
# Clip 1: Bathymetry without Ferry
clip_ferry = clip_shapefiles (bathymetry, ferry_routes_buffer, bathy_no_ferry)

In [ ]:
# Clip 2: Output without shipwrecks
clip_shipwrecks = clip_shapefiles (clip_ferry, shipwrecks_buffer, bathy_no_shipwrecks_ferry)

In [ ]:
# Clip 3: Output without SAC
clip_sac = clip_shapefiles (clip_shipwrecks, SAC, bathy_no_SAC_shipwrecks_ferry)

In [ ]:
# Clip 4: Output without SPA
clip_spa = clip_shapefiles (clip_sac, SPA, bathy_no_SPA_SAC_shipwrecks_ferry)

In [3]:
# Clip 5: Output without restricted zones
clip_restricted = clip_shapefiles (clip_spa, restricted_zones, suitable_area)

NameError: name 'clip_shapefiles' is not defined